In [2]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum
import pandas as pd
from collections import OrderedDict

In [39]:
# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = (4.44/18*1000)*(aw+bw)+198*(aj+bj)+222*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=14400, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply constraints
con4 = m.addConstr(aw+aj+ap<=600, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=2400, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Solve
m.optimize()

In [45]:
# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [(4.44/18*1000), 198, 222, (4.44/18*1000), 198, 222]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [14400, 1000, 2000, 600, 2400, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x0d2efda0
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 1e+04]
Presolve removed 5 rows and 3 columns
Presolve time: 0.02s
Presolved: 2 rows, 3 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4000000e+05   9.624497e+02   0.000000e+00      0s
       2    6.7606667e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.760666667e+05

Profit contribution: 676067

Decision variables:
aw = 525
aj = 75
ap = 0
bw = 175
bj = 225
bp = 2000


  Name  Final Val

## Question 2-4

In [131]:
# Model
m_2 = gp.Model("RBC_2")

# Create decision variables for tomatoes usage
aw = m_2.addVar(name="aw")
aj = m_2.addVar(name="aj")
ap = m_2.addVar(name="ap")
bw = m_2.addVar(name="bw")
bj = m_2.addVar(name="bj")
bp = m_2.addVar(name="bp")
AAw = m_2.addVar(name="AAw")
AAp = m_2.addVar(name="AAp")
AAj = m_2.addVar(name="AAj")

adDemandW = 0
adDemandJ = 0
adDemandP = 0

costAA = 255
#adDemandW = (5000*18)/1000
#adDemandJ = (5000*20)/1000
#adDemandP = (5000*25)/1000

obj_2 = (4.44/18*1000)*(AAw+aw+bw)+198*(AAj+aj+bj)+222*(AAp+ap+bp)-costAA*(AAw+AAj+AAp)
m_2.setObjective(obj_2, GRB.MAXIMIZE)

# Demand constraints
con11 = m_2.addConstr(aw+bw+AAw<=14400+adDemandW, name='w_dem')
con22 = m_2.addConstr(aj+bj+AAj<=1000+adDemandJ, name='j_dem')
con33 = m_2.addConstr(ap+bp+AAp<=2000+adDemandP, name='p_dem')

# Supply constraints
con44 = m_2.addConstr(aw+aj+ap<=600, name='a_sup')
con55 = m_2.addConstr(bw+bj+bp<=2400, name='b_sup')
con66 = m_2.addConstr(AAw+AAj+AAp<=80, name='AA_sup')

# Quality constraints
con77 = m_2.addConstr(9*aw+9*AAw+5*bw>=8*(aw+bw+AAw), name='w_qual')
con88 = m_2.addConstr(9*aj+9*AAj+5*bj>=6*(aj+bj+AAj), name='j_qual')

# Solve
m_2.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m_2.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m_2.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var_2 = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp', 'AAw','AAj','AAp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x, AAw.x, AAj.x, AAp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC, AAw.RC, AAj.RC, AAp.RC]),
    ('Obj Coeff', [(4.44/18*1000), 198, 222, (4.44/18*1000), 198, 222,(4.44/18*1000)-costAA,198-costAA,222-costAA]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp, AAw.SAObjUp, AAj.SAObjUp, AAp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow, AAw.SAObjLow, AAj.SAObjLow, AAp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint_2 = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'AA_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con11.Pi, con22.Pi, con33.Pi, con44.Pi, con55.Pi, con66.Pi, con77.Pi, con88.Pi]),
    ('RHS Coeff', [14400+adDemandW, 1000+adDemandJ, 2000+adDemandP, 600, 2400, 80, 0, 0]),
    ('Slack', [con11.Slack, con22.Slack, con33.Slack, con44.Slack, con55.Slack, con66.Slack, con77.Slack, con88.Slack]),
    ('Upper Range', [con11.SARHSUp, con22.SARHSUp, con33.SARHSUp, con44.SARHSUp, con55.SARHSUp, con66.SARHSUp, con77.SARHSUp, con88.SARHSUp]),
    ('Lower Range',
     [con11.SARHSLow, con22.SARHSLow, con33.SARHSLow, con44.SARHSLow, con55.SARHSLow, con66.SARHSLow, con77.SARHSUp, con88.SARHSUp])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var_2))
print('\n')
print(pd.DataFrame.from_dict(constraint_2))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 9 columns and 24 nonzeros
Model fingerprint: 0x5341bb8a
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [8e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 1e+04]
Presolve removed 3 rows and 3 columns
Presolve time: 0.02s
Presolved: 5 rows, 6 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.7304491e+05   1.816474e+02   0.000000e+00      0s
       3    6.8338833e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds (0.00 work units)
Optimal objective  6.833883333e+05

Profit contribution: 683388

Decision variables:
aw = 581.875
aj = 18.125
ap = 0
bw = 220.625
bj = 54.375
bp = 2125
AAw

## Question 5

In [101]:
# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
#aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
#bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = (4.44/18*1000)*(aw+bw) + 222*(ap+bp) - 100000
#199*(aj+bj)+
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=14400, name='w_dem')
#con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply constraints
con4 = m.addConstr(aw+ap<=600, name='a_sup')
con5 = m.addConstr(bw+bp<=2400, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
#con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aj', 'ap', 'bj', 'bp']),
    ('Final Value', [aw.x, ap.x,  bw.x, bp.x]),
    ('Reduced Cost', [aw.RC, ap.RC, bw.RC, bp.RC]),
    ('Obj Coeff', [(4.44/18*1000), (4.44/18*1000), 198, 198]),
    ('Upper Range', [aw.SAObjUp, ap.SAObjUp,  bw.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, ap.SAObjLow,  bw.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual']),
    ('Shadow Price', [con1.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi]),
    ('RHS Coeff', [14400, 2000, 600, 2400, 0]),
    ('Slack', [con1.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack]),
    ('Upper Range', [con1.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 4 columns and 10 nonzeros
Model fingerprint: 0x76daf764
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 1e+04]
Presolve removed 5 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.4133333e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.413333333e+05

Profit contribution: 541333

Decision variables:
aw = 600
ap = 0
bw = 200
bp = 2000


  Name  Final Value  Reduced Cost   Obj Coeff  Upper Range   Lower Range
0   aj        600.0      0.0

In [230]:
# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
#ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
#bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 198*(aj+bj)+(4.44/18*1000)*(aw+bw) - 100000
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=14400, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
#con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply constraints
con4 = m.addConstr(aw+aj<=600, name='a_sup')
con5 = m.addConstr(bw+bj<=2400, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'ap', 'bw', 'bp']),
    ('Final Value', [aw.x, aj.x,  bw.x, bj.x]),
    ('Reduced Cost', [aw.RC, aj.RC, bw.RC, bj.RC]),
    ('Obj Coeff', [(4.44/18*1000),198, (4.44/18*1000), 198]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp,  bw.SAObjUp, bj.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow,  bw.SAObjLow, bj.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'a_sup', 'b_sup', 'w_qual','j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [14000,1000, 600, 2400, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con4.Slack, con5.Slack, con6.Slack,  con7.Slack]),
    ('Upper Range', [con2.SARHSUp, con2.SARHSUp, con4.SARHSUp, con5.SARHSUp,con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con4.SARHSLow, con5.SARHSLow,con5.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x8bec675b
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 1e+04]
Presolve removed 6 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1311111e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.131111111e+05

Profit contribution: 213111

Decision variables:
aw = 350
aj = 250
bw = 116.667
bj = 750


  Name  Final Value  Reduced Cost   Obj Coeff  Upper Range   Lower Range
0   aw   350.000000    

## Question 6

In [3]:
# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVars(3,name="aw")
aj = m.addVars(3, name="aj")
ap = m.addVars(3, name="ap")
bw = m.addVars(3, name="bw")
bj = m.addVars(3, name="bj")
bp = m.addVars(3, name="bp")

#Decision Variable for Order Quantity
order_quant = m.addVar(name="order_quantity", ub=13000)

prob = [0.25,0.5,0.25]
yield_pred = [0.6,0.5,0.2]

objs = [(4.44/18*1000)*(aw[i]+bw[i]) + 198*(aj[0]+bj[i]) + 222*(ap[i]+bp[i]) for i in range(3)]

m.setObjective(quicksum(prob[i]*objs[i] for i in range(3)) - 200*order_quant, GRB.MAXIMIZE)

#Demand Constraints
con1 = m.addConstrs((aw[i]+bw[i]<=14400 for i in range(3)), name="w_dem")
con2 = m.addConstrs((aj[i]+bj[i]<=1000 for i in range(3)), name="j_dem")
con3 = m.addConstrs((ap[i]+bp[i]<=2000 for i in range(3)), name="p_dem")

# Quality constraints
con4 = m.addConstrs((9*aw[i] + 5*bw[i] >= 8*(aw[i] + bw[i]) for i in range(3)), name='w_qual')
con5 = m.addConstrs((9*aj[i] + 5*bj[i] >= 8*(aj[i] + bj[i]) for i in range(3)), name='j_qual')

#Supply Constraints
con6 = m.addConstrs((aw[i]+aj[i]+ap[i] <= yield_pred[i]*order_quant for i in range(3)), name='a_sup')
con7 = m.addConstrs((bw[i]+bj[i]+bp[i] <= (1-yield_pred[i])*order_quant for i in range(3)), name='b_sup')

m.setParam('OutputFlag', 0)

# Solve
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-17


In [4]:
# Print optimal value of the objective function
print('\nExpected Net Profit: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))



Expected Net Profit: 234641

Decision variables:
aw[0] = 636.364
aw[1] = 1363.64
aw[2] = 545.455
aj[0] = 1000
aj[1] = 0
aj[2] = 0
ap[0] = 0
ap[1] = 0
ap[2] = 0
bw[0] = 212.121
bw[1] = 454.545
bw[2] = 181.818
bj[0] = 0
bj[1] = 0
bj[2] = 0
bp[0] = 878.788
bp[1] = 909.091
bp[2] = 2000
order_quantity = 2727.27


In [5]:
# Create table for decision variables' sensitivity analysis
decision_vars = {}
for i in range(3):
    dec_var = OrderedDict([
        ('Name', ['aw_'f'{i}', 'aj_'f'{i}', 'ap_'f'{i}','bw_'f'{i}','bj_'f'{i}', 'bp_'f'{i}','order_quant']),
        ('Final Value', [aw[i].x, aj[i].x, ap[i].x, bw[i].x, bj[i].x, bp[i].x, order_quant.x]),
        ('Reduced Cost', [aw[i].RC, aj[i].RC, ap[i].RC, bw[i].RC, bj[i].RC, bp[i].RC, order_quant.RC]),
        ('Obj Coeff', [(4.44/18*1000), 198, 222, (4.44/18*1000), 198, 222, -200]),
        ('Upper Range', [aw[i].SAObjUp, aj[i].SAObjUp, ap[i].SAObjUp, bw[i].SAObjUp, bj[i].SAObjUp, bp[i].SAObjUp, order_quant.SAObjUp]),
        ('Lower Range', [aw[i].SAObjLow, aj[i].SAObjLow, ap[i].SAObjLow, bw[i].SAObjLow, bj[i].SAObjLow, bp[i].SAObjLow, order_quant.SAObjLow])
    ])
    decision_vars['decision_vars_model_'f'{i}'] = dec_var

for key,value in decision_vars.items():
    print(key, ':\n')
    print(pd.DataFrame.from_dict(value))
    print('\n')

decision_vars_model_0 :

          Name  Final Value  Reduced Cost   Obj Coeff  Upper Range  \
0         aw_0   636.363636      0.000000  246.666667    68.166667   
1         aj_0  1000.000000      0.000000  198.000000          inf   
2         ap_0     0.000000     -8.222222  222.000000    63.722222   
3         bw_0   212.121212      0.000000  246.666667    81.166667   
4         bj_0     0.000000   -140.277778  198.000000   189.777778   
5         bp_0   878.787879      0.000000  222.000000    61.666667   
6  order_quant  2727.272727      0.000000 -200.000000  -196.100000   

   Lower Range  
0    53.444444  
1    63.722222  
2         -inf  
3    55.500000  
4         -inf  
5    -0.000000  
6  -232.400000  


decision_vars_model_1 :

          Name  Final Value  Reduced Cost   Obj Coeff  Upper Range  \
0         aw_1  1363.636364      0.000000  246.666667   131.133333   
1         aj_1     0.000000   -127.444444  198.000000   127.444444   
2         ap_1     0.000000    -16.444444

In [7]:
constraints = {}
for i in range(3):
    con = OrderedDict([
        ('Name', ['w_dem_'f'{i}', 'j_dem_'f'{i}', 'p_dem_'f'{i}', 'w_qual_'f'{i}', 'p_qual_'f'{i}', 'a_sup_'f'{i}', 'b_sup_'f'{i}']),
        ('Shadow Price', [con1[i].Pi, con2[i].Pi, con3[i].Pi, con4[i].Pi, con5[i].Pi, con6[i].Pi, con7[i].Pi]),
        ('RHS Coeff', [14400, 1000, 2000, 0, 0, yield_pred[i]*order_quant.x, (1-yield_pred[i])*order_quant.x]),
        ('Slack', [con1[i].Slack, con2[i].Slack, con3[i].Slack, con4[i].Slack, con5[i].Slack, con6[i].Slack, con7[i].Slack]),
        ('Upper Range', [con1[i].SARHSUp, con2[i].SARHSUp, con3[i].SARHSUp, con4[i].SARHSUp, con5[i].SARHSUp, con6[i].SARHSUp, con7[i].SARHSUp]),
        ('Lower Range',
        [con1[i].SARHSLow, con2[i].SARHSLow, con3[i].SARHSLow, con4[i].SARHSLow, con5[i].SARHSLow, con6[i].SARHSLow, con7[i].SARHSLow])
        ])
    constraints['Constraints_for_Model_'f'{i}'': '] = con

    for key,value in constraints.items():
        print(key, ':\n')
        print(pd.DataFrame.from_dict(value))
        print('\n')

Constraints_for_Model_0:  :

       Name  Shadow Price     RHS Coeff         Slack  Upper Range  \
0   w_dem_0      0.000000  14400.000000  13551.515152          inf   
1   j_dem_0    134.277778   1000.000000      0.000000  1636.363636   
2   p_dem_0      0.000000   2000.000000   1121.212121          inf   
3  w_qual_0     -2.055556      0.000000      0.000000   636.363636   
4  p_qual_0      0.000000      0.000000  -1000.000000  1000.000000   
5   a_sup_0     63.722222   1636.363636      0.000000  2636.363636   
6   b_sup_0     55.500000   1090.909091      0.000000  1121.212121   

   Lower Range  
0   848.484848  
1     0.000000  
2   878.787879  
3 -2636.363636  
4         -inf  
5  -636.363636  
6  -878.787879  


Constraints_for_Model_0:  :

       Name  Shadow Price     RHS Coeff         Slack  Upper Range  \
0   w_dem_0      0.000000  14400.000000  13551.515152          inf   
1   j_dem_0    134.277778   1000.000000      0.000000  1636.363636   
2   p_dem_0      0.000000   2000.